In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from IPython.core.display import display, HTML
import os
%matplotlib inline

In [2]:
# Notebook Styling 
sns.set()
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format',lambda x: '%.5f' % x)

In [5]:
CSV_PATH = os.path.join('data', 'fivethirtyeight', 'dem_candidates_2018_prim.csv')
df = pd.read_csv(CSV_PATH, encoding='latin1', parse_dates=['Race Primary Election Date'])
df.head()

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Gender,Partisan Lean,Primary %,Won Primary,Race,Veteran?,LGBTQ?,Elected Official?,Self-Funder?,STEM?,Obama Alum?
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,3.42000,No,Nonwhite,Yes,No,No,No,No,No
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,1.74000,No,White,No,Yes,No,No,No,No
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,3.27000,No,White,Yes,No,No,No,No,No
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,8.00000,No,Nonwhite,Yes,No,Yes,No,No,No
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Female,-28.88000,28.98000,No,White,No,No,Yes,No,No,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 20 columns):
Candidate                     811 non-null object
State                         811 non-null object
District                      811 non-null object
Office Type                   811 non-null object
Race Type                     811 non-null object
Race Primary Election Date    811 non-null datetime64[ns]
Primary Status                807 non-null object
Primary Runoff Status         811 non-null object
General Status                807 non-null object
Gender                        811 non-null object
Partisan Lean                 811 non-null float64
Primary %                     801 non-null float64
Won Primary                   785 non-null object
Race                          655 non-null object
Veteran?                      800 non-null object
LGBTQ?                        800 non-null object
Elected Official?             800 non-null object
Self-Funder?                  811 n

In [7]:
df.isnull().sum()

Candidate                       0
State                           0
District                        0
Office Type                     0
Race Type                       0
Race Primary Election Date      0
Primary Status                  4
Primary Runoff Status           0
General Status                  4
Gender                          0
Partisan Lean                   0
Primary %                      10
Won Primary                    26
Race                          156
Veteran?                       11
LGBTQ?                         11
Elected Official?              11
Self-Funder?                    0
STEM?                          11
Obama Alum?                     1
dtype: int64

We can see the counts of missing values in the printout above. Understandably, race is the most common missing value. Per the list of unique values below, we see that the race category has been grouped as 'white' and 'nonwhite'.

In [17]:
df['Race'].unique()

array(['Nonwhite', 'White', nan], dtype=object)

In [16]:
sum(df['Race Primary Election Date'] > pd.datetime.now())

0

In [9]:
df.columns

Index(['Candidate', 'State', 'District', 'Office Type', 'Race Type',
       'Race Primary Election Date', 'Primary Status', 'Primary Runoff Status',
       'General Status', 'Gender', 'Partisan Lean', 'Primary %', 'Won Primary',
       'Race', 'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?',
       'STEM?', 'Obama Alum?'],
      dtype='object')

Ok, so we see that all of the included primaries have already occured.

In [18]:
df.head()

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Gender,Partisan Lean,Primary %,Won Primary,Race,Veteran?,LGBTQ?,Elected Official?,Self-Funder?,STEM?,Obama Alum?
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,3.42000,No,Nonwhite,Yes,No,No,No,No,No
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,1.74000,No,White,No,Yes,No,No,No,No
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,3.27000,No,White,Yes,No,No,No,No,No
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Male,-28.88000,8.00000,No,Nonwhite,Yes,No,Yes,No,No,No
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,2018-06-05,Lost,None,None,Female,-28.88000,28.98000,No,White,No,No,Yes,No,No,No


In [32]:
cols = ['Office Type', 'Race Type', 'Primary Status', 'Primary Runoff Status',
       'General Status', 'Gender',  'Won Primary',
       'Race', 'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?',
       'STEM?', 'Obama Alum?']
for feature in cols:
    print('Feature: {:30s}, Number of Unique Vals: {},  Unique Values: {}'
          .format(feature, len(df[feature].unique()), df[feature].unique()))

Feature: Office Type                   , Number of Unique Vals: 3,  Unique Values: ['Governor' 'Representative' 'Senator']
Feature: Race Type                     , Number of Unique Vals: 2,  Unique Values: ['Regular' 'Special']
Feature: Primary Status                , Number of Unique Vals: 3,  Unique Values: ['Lost' 'Advanced' nan]
Feature: Primary Runoff Status         , Number of Unique Vals: 4,  Unique Values: ['None' 'Lost' 'Advanced' 'On the Ballot']
Feature: General Status                , Number of Unique Vals: 3,  Unique Values: ['None' 'On the Ballot' nan]
Feature: Gender                        , Number of Unique Vals: 2,  Unique Values: ['Male' 'Female']
Feature: Won Primary                   , Number of Unique Vals: 3,  Unique Values: ['No' 'Yes' nan]
Feature: Race                          , Number of Unique Vals: 3,  Unique Values: ['Nonwhite' 'White' nan]
Feature: Veteran?                      , Number of Unique Vals: 3,  Unique Values: ['Yes' 'No' nan]
Feature: LGBTQ?   

From a quick inspection of the data, we see that all of the above features are categorical. 

2 of the features ('Office Type' and 'Primary Runoff Status') are multicategory features, while the remainder are binary features, although most of them have missing values that need to be addressed.

In [34]:
df[df['Race'].isnull()]

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Gender,Partisan Lean,Primary %,Won Primary,Race,Veteran?,LGBTQ?,Elected Official?,Self-Funder?,STEM?,Obama Alum?
271,Michael Brown,MD,U.S. House Maryland District 1,Representative,Regular,2018-06-26,Lost,None,None,Male,-29.66000,15.00000,No,NaN,NaN,NaN,NaN,No,NaN,NaN
289,Craig Olson,ME,U.S. House Maine District 2,Representative,Regular,2018-06-12,Lost,None,None,Male,-8.13000,8.83000,No,NaN,No,No,No,No,No,No
290,Jared Golden,ME,U.S. House Maine District 2,Representative,Regular,2018-06-12,Advanced,None,On the Ballot,Male,-8.13000,46.42000,Yes,NaN,Yes,No,Yes,No,No,No
291,Lucas St. Clair,ME,U.S. House Maine District 2,Representative,Regular,2018-06-12,Lost,None,None,Male,-8.13000,39.24000,No,NaN,No,No,No,No,No,No
292,Zak Ringelstein,ME,U.S. Senate Maine,Senator,Regular,2018-06-12,Advanced,None,On the Ballot,Male,3.51000,100.00000,Yes,NaN,No,No,No,No,No,No
298,Michael McCarthy (Michigan),MI,U.S. House Michigan District 10,Representative,Regular,2018-08-07,Lost,None,None,Male,-29.58000,26.90000,No,NaN,NaN,NaN,NaN,No,NaN,No
338,Ed Andres,MO,U.S. House Missouri District 6,Representative,Regular,2018-08-07,Lost,None,None,Male,-31.61000,27.70000,No,NaN,NaN,NaN,NaN,No,NaN,No
343,Kenneth Hatfield,MO,U.S. House Missouri District 7,Representative,Regular,2018-08-07,Lost,None,None,Male,-46.13000,22.30000,No,NaN,NaN,NaN,NaN,No,NaN,No
344,Vincent Jennings,MO,U.S. House Missouri District 7,Representative,Regular,2018-08-07,Lost,None,None,Male,-46.13000,15.40000,No,NaN,NaN,NaN,NaN,No,NaN,No
346,Randy Wadkins,MS,U.S. House Mississippi District 1,Representative,Regular,2018-06-05,Advanced,None,On the Ballot,Male,-33.51000,100.00000,Yes,NaN,No,No,No,No,Yes,No


In [57]:
no_dupes = len(df.loc[:,['Candidate','District']].drop_duplicates())
print('Without duplicated candidate names, there are {} entries in the data set'.format(no_dupes))
print('With duplicated candidate names, there are    {} entries in the data set'.format(len(df)))

Without duplicated candidate names, there are 801 entries in the data set
With duplicated candidate names, there are    811 entries in the data set


From the printout, we can see that either 10 candidates match on both name and district. It seems very strange that multiple people would have the same name and be running in the same district. This deserves further inspection.

Per inspection, we see that these cases involve people running in both a regular and special election.  

In [58]:
df[df.loc[:,['Candidate','District']].duplicated(keep=False)]

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Gender,Partisan Lean,Primary %,Won Primary,Race,Veteran?,LGBTQ?,Elected Official?,Self-Funder?,STEM?,Obama Alum?
304,Bill Wild,MI,U.S. House Michigan District 13,Representative,Regular,2018-08-07,Lost,None,None,Male,60.72000,14.00000,No,White,No,No,Yes,No,No,No
305,Bill Wild,MI,U.S. House Michigan District 13,Representative,Special,2018-08-07,Lost,None,None,Male,60.72000,15.20000,No,White,No,No,Yes,No,No,No
306,Brenda Jones,MI,U.S. House Michigan District 13,Representative,Special,2018-08-07,Advanced,None,On the Ballot,Female,60.72000,37.70000,Yes,Nonwhite,No,No,Yes,No,No,No
307,Brenda Jones,MI,U.S. House Michigan District 13,Representative,Regular,2018-08-07,Lost,None,None,Female,60.72000,29.20000,No,Nonwhite,No,No,Yes,No,No,No
309,Ian Conyers,MI,U.S. House Michigan District 13,Representative,Regular,2018-08-07,Lost,None,None,Male,60.72000,6.30000,No,Nonwhite,No,No,Yes,No,No,No
310,Ian Conyers,MI,U.S. House Michigan District 13,Representative,Special,2018-08-07,Lost,None,None,Male,60.72000,11.20000,No,Nonwhite,No,No,Yes,No,No,No
311,Rashida Tlaib,MI,U.S. House Michigan District 13,Representative,Regular,2018-08-07,Advanced,None,On the Ballot,Female,60.72000,33.20000,Yes,Nonwhite,No,No,Yes,No,No,Yes
312,Rashida Tlaib,MI,U.S. House Michigan District 13,Representative,Special,2018-08-07,Lost,None,None,Female,60.72000,35.90000,No,Nonwhite,No,No,Yes,No,No,Yes
500,Danny O'Connor,OH,U.S. House Ohio District 12,Representative,Regular,2018-05-08,Advanced,None,On the Ballot,Male,-13.63000,40.52000,Yes,NaN,No,No,Yes,No,No,No
501,Danny O'Connor,OH,U.S. House Ohio District 12,Representative,Special,2018-05-08,Advanced,None,On the Ballot,Male,-13.63000,40.94000,Yes,NaN,No,No,Yes,No,No,No


I want to see if race is an important feature, so I'll do some leg work to fill in the data. Race (or at least racial appearance) can be identified by sight, and people who run for office tend to be photographed. So far, I've given up on finding photos of 
* Michael Brown (MD)
* Jack Schofield Jr. (NV)

In [134]:
df[df['Race'].isnull()]

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Gender,Partisan Lean,Primary %,Won Primary,Race,Veteran?,LGBTQ?,Elected Official?,Self-Funder?,STEM?,Obama Alum?
271,Michael Brown,MD,U.S. House Maryland District 1,Representative,Regular,2018-06-26,Lost,None,None,Male,-29.66000,15.00000,No,NaN,NaN,NaN,NaN,No,NaN,NaN
428,Jack Schofield Jr.,NV,U.S. House Nevada District 2,Representative,Regular,2018-06-12,Lost,None,None,Male,-13.78000,7.50000,No,NaN,No,No,No,No,No,No
431,Rick Shepherd,NV,U.S. House Nevada District 2,Representative,Regular,2018-06-12,Lost,None,None,Male,-13.78000,21.28000,No,NaN,No,No,No,No,Yes,No
432,Vance Alm,NV,U.S. House Nevada District 2,Representative,Regular,2018-06-12,Lost,None,None,Male,-13.78000,13.22000,No,NaN,Yes,No,No,No,No,No
433,Eric Stoltz,NV,U.S. House Nevada District 3,Representative,Regular,2018-06-12,Lost,None,None,Male,-3.08000,7.24000,No,NaN,No,No,No,No,No,No
434,Guy Pinjuv,NV,U.S. House Nevada District 3,Representative,Regular,2018-06-12,Lost,None,None,Male,-3.08000,3.50000,No,NaN,No,No,No,No,Yes,No
435,Jack Love,NV,U.S. House Nevada District 3,Representative,Regular,2018-06-12,Lost,None,None,Male,-3.08000,5.80000,No,NaN,No,No,No,No,No,No
436,Michael Weiss,NV,U.S. House Nevada District 3,Representative,Regular,2018-06-12,Lost,None,None,Male,-3.08000,8.18000,No,NaN,No,No,No,No,Yes,No
437,Richard Hart,NV,U.S. House Nevada District 3,Representative,Regular,2018-06-12,Lost,None,None,Male,-3.08000,4.85000,No,NaN,No,No,No,No,No,No
438,Steve Schiffman,NV,U.S. House Nevada District 3,Representative,Regular,2018-06-12,Lost,None,None,Male,-3.08000,3.51000,No,NaN,No,No,No,No,No,No


In [133]:
df.loc[(df['Candidate'] == 'Clint Koble') & (df['State'] == 'NV'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jesse Hurley') & (df['State'] == 'NV'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Patrick Fogarty') & (df['State'] == 'NV'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Patrick Fogarty') & (df['State'] == 'NV'), 'Race'] = 'White'

In [129]:


df.loc[(df['Candidate'] == 'Danny O\'Connor') & (df['State'] == 'OH'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'John Russell') & (df['State'] == 'OH'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jackie Patton') & (df['State'] == 'OH'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Zach Scott') & (df['State'] == 'OH'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Ed Albertson') & (df['State'] == 'OH'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Doug Wilson') & (df['State'] == 'OH'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Craig Olson') & (df['State'] == 'ME'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jared Golden') & (df['State'] == 'ME'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Lucas St. Clair') & (df['State'] == 'ME'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Zak Ringelstein') & (df['State'] == 'ME'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Michael McCarthy (Michigan)') & (df['State'] == 'MI'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Ed Andres') & (df['State'] == 'MO'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Kenneth Hatfield') & (df['State'] == 'MO'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Vincent Jennings') & (df['State'] == 'MO'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Randy Wadkins') & (df['State'] == 'MS'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Michael Aycox') & (df['State'] == 'MS'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Michael Evans') & (df['State'] == 'MS'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jensen Bohren') & (df['State'] == 'MS'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Grant Kier') & (df['State'] == 'MT'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jared Pettinato') & (df['State'] == 'MT'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'John Heenan') & (df['State'] == 'MT'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'John Meyer') & (df['State'] == 'MT'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Kathleen Williams') & (df['State'] == 'MT'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Lynda Moss') & (df['State'] == 'MT'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Mac Schneider') & (df['State'] == 'ND'), 'Race'] = 'White'

df.loc[(df['Candidate'] == 'Dennis Crawford') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jessica McClure') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Brad Ashford') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Kara Eastman') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Paul Theobald') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Chris Janicek') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Frank Svoboda') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Jane Raybould') & (df['State'] == 'NE'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Larry Marvin') & (df['State'] == 'NE'), 'Race'] = 'White'

df.loc[(df['Candidate'] == 'Alison Heslin') & (df['State'] == 'NJ'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Mark Washburne') & (df['State'] == 'NJ'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Mikie Sherrill') & (df['State'] == 'NJ'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Mitchell Cobert') & (df['State'] == 'NJ'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Nathan Kleinman') & (df['State'] == 'NJ'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Josh Welle') & (df['State'] == 'NJ'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Tom Malinowski') & (df['State'] == 'NJ'), 'Race'] = 'White'

df.loc[(df['Candidate'] == 'Madeline Hildebrandt') & (df['State'] == 'NM'), 'Race'] = 'White'

df.loc[(df['Candidate'] == 'Chris Giunchigliani') & (df['State'] == 'NV'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'David Jones') & (df['State'] == 'NV'), 'Race'] = 'Nonwhite'
df.loc[(df['Candidate'] == 'John Bonaventura') & (df['State'] == 'NV'), 'Race'] = 'White'
df.loc[(df['Candidate'] == 'Steve Sisolak') & (df['State'] == 'NV'), 'Race'] = 'White'

In [73]:
print('Feature: {:30s}, Number of Unique Vals: {},  Unique Values: {}'
          .format('Race', len(df['Race'].unique()), df['Race'].unique()))

Feature: Race                          , Number of Unique Vals: 3,  Unique Values: ['Nonwhite' 'White' nan]


In [71]:
# Pre Adjustment
df['Race'].isnull().sum()

Candidate                       0
State                           0
District                        0
Office Type                     0
Race Type                       0
Race Primary Election Date      0
Primary Status                  4
Primary Runoff Status           0
General Status                  4
Gender                          0
Partisan Lean                   0
Primary %                      10
Won Primary                    26
Race                          154
Veteran?                       11
LGBTQ?                         11
Elected Official?              11
Self-Funder?                    0
STEM?                          11
Obama Alum?                     1
dtype: int64

In [108]:
# Post Adjustment
df['Race'].isnull().sum()

127